# Codelab for Convolutional Neural Net

All rights reserved.<br \>
This material cannot be published, rewritten or redistributed in whole or part without the authors' written permission.

**Convolutional Neural Net** will be abbreviated to **ConvNet** in the following sections.

### Index
1. [Pre-processing](#Pre-processing)
2. [Recipes for ConvNet](#Recipes-Exclusively-for-Convolutional-Neural-Nets)
    1. [Network Structure](#Network-Structure)
        1. [Activity 1](#Tutorial/workshop-activity-1)
    2. [Pooling](#Pooling)
        1. [Activity 2](#Tutorial/workshop-activity-2)
3. [Define Training Procedure](#Define-Training-Procedure)     
4. [Start Training](#Start-Training) and [Observe Training Process](#Observe-Training-Process)
5. [Testing](#Define-Testing-Procedure)

During tutorial/workshop, attendees will be separated into three groups. Each group will be conducting different activities.<br />

Activity: "Cell" $\rightarrow$ "Run All Below" for testing the enviroment.



In [12]:
################################################################
#
# All rights reserved.
#
# This is a codelab for Convolutional Neural Net.
# Details include:
#   - Pre-process dataset
#   - Elaborate recipes
#   - Define training procedures
#   - Train and test models
#   - Observe metrics
#
################################################################
from __future__ import print_function

import keras.callbacks as cb
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import SGD
from keras.regularizers import l1, l2
from keras.utils import np_utils

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import time

## Pre-processing

Pre-processing for ConvNet is slightly different than that of FNNs. Here, you need to transform the data into a 3-D tensor with dimensions [width] $\times$ [height] $\times$  [channel].

Width and height are from a image.
Each channel represents a color.

Common techniques for data augmentation:
1. Cropping
2. Random cropping
3. Rotation
4. Add random noises


In [13]:
def PreprocessDataset():
    # Load dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # Set numeric type
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # Normalize value to [0, 1]
    x_train /= 255
    x_test /= 255
    # Transform lables to one-hot
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    
    # Reshape: here x_train is re-shaped to [width] × [height] x [channel]
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    return [x_train, x_test, y_train, y_test]

x_train, x_test, y_train, y_test = PreprocessDataset()

## Recipes Exclusively for Convolutional Neural Nets

### Convolutional Layer
Network structure refers to components of a convnet.

Ingredients:
1. **Filters**: define the size of a filter, and the total number of filters in a convolutional layer.
2. **Depth** represents the number of layers, usually 10-30.
3. **Stride** defines the step at which the filter moves.
4. **Padding** of the input data, either no padding or zero-padding.

How to calculate the number of parameters

Functionalityies of convolutional layers: 
- Capture patterns
- Share parameters (otherwise too many parameters)

### Pooling
Pooling layer reduces the output of the previous layer.

max-k pool (usually k = 1)


In [14]:
num_filters = 16
filter_size = 3

def DefineModel():
    model = Sequential()
    # First conv layer with filters of 3x3 pixels; also need to specify input shape.
    # The number of filters is set to 16 for shorter runtime.
    # Increase the number of filters for better accuracy.
    # Strides are defined as in subsample=(1, 1)
    model.add(Convolution2D(num_filters, filter_size, filter_size, border_mode='valid', input_shape=(28,28,1)))
    # Max pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # The last layer has the same dimension as the number of classes
    model.add(Flatten())
    model.add(Dense(10))
    # For classification, the activation is softmax
    model.add(Activation('softmax'))
    # Define optimizer
    optmzr = SGD(lr=0.1, clipnorm=5.)
    # Define loss function = cross entropy
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])

    return model

## Define Training Procedure

In [15]:
def TrainModel(data=None, epochs=20, batch=256):
    start_time = time.time()
    model = DefineModel()
    if data is None:
        print("Must provide data.")
        return
    x_train, x_test, y_train, y_test = data
    print(x_train.shape)
    print(x_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    print(x_train[1].shape)
    print('Start training.')
    history = model.fit(x_train, y_train, nb_epoch=epochs, batch_size=batch,
              validation_data=(x_test, y_test), verbose=1)
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

## Start Training

In [16]:
trained_model, training_history = TrainModel(data=[x_train, x_test, y_train, y_test])

OverflowError: Range exceeds valid bounds

## Observe Training Process
Plot the variation of loss during each batch

In [ ]:
def PlotHistory(train_value, test_value, value_is_loss_or_acc):
    f, ax = plt.subplots()
    ax.plot([None] + train_value, 'o-')
    ax.plot([None] + test_value, 'x-')
    # Plot legend and use the best location automatically: loc = 0.
    ax.legend(['Train ' + value_is_loss_or_acc, 'Validation ' + value_is_loss_or_acc], loc = 0) 
    ax.set_title('Training/Validation ' + value_is_loss_or_acc + ' per Epoch')
    ax.set_xlabel('Epoch')
    ax.set_ylabel(value_is_loss_or_acc)  
    
PlotHistory(training_history.history['loss'], training_history.history['val_loss'], 'Loss')
PlotHistory(training_history.history['acc'], training_history.history['val_acc'], 'Accuracy')

## Testing

In [ ]:
def TestModel(model=None, data=None):
    if model is None:
        print("Must provide a trained model.")
        return
    if data is None:
        print("Must provide data.")
        return
    x_test, y_test = data
    scores = model.evaluate(x_test, y_test)
    return scores

## Test the trained model

In [ ]:
test_score = TestModel(model=trained_model, data=[x_test, y_test])
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

## Visualize Convolutional Layers

In [ ]:
def ShowConvolutionOutput(input_data, show_pic_idx):
    from keras import backend as K
    first_conv = K.function(trained_model.inputs, [trained_model.layers[1].output])
    first_conv_out = first_conv([input_data])
    fig, axes = plt.subplots(int(num_filters / 4), 4,figsize=(8, 8))
    for filter_num in range(num_filters):
        fig_x, fig_y = divmod(filter_num, 4)
        if fig_y == 0: fig_x -= 1
        axes[fig_x][fig_y].imshow(first_conv_out[0][show_pic_idx][:,:,filter_num])
        plt.setp(axes[fig_x][fig_y].get_xticklabels(), visible=False)
        plt.setp(axes[fig_x][fig_y].get_yticklabels(), visible=False)
    fig.tight_layout(pad=0)
    plt.show()

def ShowInputImage(data):
    plot = plt.figure()
    plot.set_size_inches(2,2)
    plt.imshow(np.reshape(-data, (28,28)), cmap='Greys_r')
    plt.title("Input")
    plt.axis('off')
    plt.show()

def ShowFinalOutput(input_data, samp):
    """Calculate final prediction."""
    from keras import backend as K
    # Calculate final prediction.
    last_layer = K.function(trained_model.inputs, [trained_model.layers[-1].output])
    last_layer_out = last_layer([input_data])
    print("Final prediction: " + str(np.argmax(last_layer_out[0][samp])) )

## Show Random Test Data

In [ ]:
np.random.seed(1337)
show_which_ones = np.random.randint(0, 999, 5)
for samp in show_which_ones:
    ShowInputImage(x_test[samp])
    ShowConvolutionOutput(x_test, samp)
    ShowFinalOutput(x_test, samp)